In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_231132/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 08:54:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 08:54:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/15 08:54:08 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Student table
student_data = {
    'student_id': [1, 2, 3, 4, 5],
    'student_name': ['Daniel', 'Jade', 'Stella', 'Jonathan', 'Will']
}

# Sample data for Exam table
exam_data = {
    'exam_id': [10, 10, 10, 20, 30, 30, 30, 40, 40, 40],
    'student_id': [1, 2, 3, 1, 1, 3, 4, 1, 2, 4],
    'score': [70, 80, 90, 80, 70, 80, 90, 60, 70, 80]
}

# Create DataFrame for Student and Exam tables
student_df = pd.DataFrame(student_data)
exam_df = pd.DataFrame(exam_data)


df_person = spark.createDataFrame(student_df)
df_person.createOrReplaceTempView("Student")

df_person = spark.createDataFrame(exam_df)
df_person.createOrReplaceTempView("Exam")



In [4]:
student_df

,student_id,student_name
0,1,Daniel
1,2,Jade
2,3,Stella
3,4,Jonathan
4,5,Will


In [5]:
exam_df

,exam_id,student_id,score
0,10,1,70
1,10,2,80
2,10,3,90
3,20,1,80
4,30,1,70
5,30,3,80
6,30,4,90
7,40,1,60
8,40,2,70
9,40,4,80


In [8]:
query = """
    select
        *,
        max(score) over(partition by exam_id) as max_score,
        min(score) over(partition by exam_id) as min_score,
        case
            when (score = max(score) over(partition by exam_id)) or (score = min(score) over(partition by exam_id)) then true
            else false
        end as not_quite
    from Exam e 
"""

In [9]:
spark.sql(query).show()

+-------+----------+-----+---------+---------+---------+
|exam_id|student_id|score|max_score|min_score|not_quite|
+-------+----------+-----+---------+---------+---------+
|     10|         1|   70|       90|       70|     true|
|     10|         2|   80|       90|       70|    false|
|     10|         3|   90|       90|       70|     true|
|     20|         1|   80|       80|       80|     true|
|     30|         1|   70|       90|       70|     true|
|     30|         3|   80|       90|       70|    false|
|     30|         4|   90|       90|       70|     true|
|     40|         1|   60|       80|       60|     true|
|     40|         2|   70|       80|       60|    false|
|     40|         4|   80|       80|       60|     true|
+-------+----------+-----+---------+---------+---------+



In [13]:
query_1 = f"""
    with t as (
        select
            distinct student_id
        from (
            {query}
        )
        where not_quite = true
    )
    
    select
        k.student_id,
        s.student_name
    from (
        select distinct student_id from Exam 
    ) k left join Student s on k.student_id = s.student_id
    where k.student_id not in (select student_id from t)
"""

In [14]:
spark.sql(query_1).show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         2|        Jade|
+----------+------------+

